In [37]:
import numpy as np
import pandas as pd
import itertools

In [20]:
class Card(np.ndarray):
    
    def __new__(subtype, card_val):
        
        
        
        obj = super(Card, subtype).__new__(subtype, shape=(2,), dtype=np.int)
        
        return obj
    
    def __array_finalize__(self, obj):
        
        if obj is None:
            return

In [17]:
class InfoArray(np.ndarray):

    def __new__(subtype, shape, dtype=float, buffer=None, offset=0,
                strides=None, order=None, info=None):
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        print(subtype)
        obj = super(InfoArray, subtype).__new__(subtype, shape, dtype,
                                                buffer, offset, strides,
                                                order)
        # set the new 'info' attribute to the value passed
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None: return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).
        self.info = getattr(obj, 'info', None)
        # We do not need to return anythingclass InfoArray(np.ndarray):


In [74]:
value_dict = {"2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7":7, "8": 8, "9": 9, "10": 10, "J": 11, "Q": 12, "K": 13, "A": 14}
suit_dict = {"d": 0, "c": 1, "s": 2, "h": 3}

In [75]:
num = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
suit = ["d", "c", "s", "h"]

In [76]:
deck = [n + ' ' +  s for n in num for s in suit]

In [91]:
np.meshgrid(deck_arr, deck_arr).shape

AttributeError: 'list' object has no attribute 'shape'

In [78]:
for idx, card in enumerate(deck):
    deck[idx] = np.array(
    [value_dict[card.split(' ')[0]], suit_dict[card.split(' ')[1]]])

In [81]:
deck_arr = np.array(deck)

In [88]:
deck_arr[[3, 7, 12], :] + deck_arr[[2, 3], :]

array([[2, 3],
       [3, 3],
       [5, 0]])

In [96]:
list(itertools.combinations(np.arange(1, 44), 5))

[(1, 2, 3, 4, 5),
 (1, 2, 3, 4, 6),
 (1, 2, 3, 4, 7),
 (1, 2, 3, 4, 8),
 (1, 2, 3, 4, 9),
 (1, 2, 3, 4, 10),
 (1, 2, 3, 4, 11),
 (1, 2, 3, 4, 12),
 (1, 2, 3, 4, 13),
 (1, 2, 3, 4, 14),
 (1, 2, 3, 4, 15),
 (1, 2, 3, 4, 16),
 (1, 2, 3, 4, 17),
 (1, 2, 3, 4, 18),
 (1, 2, 3, 4, 19),
 (1, 2, 3, 4, 20),
 (1, 2, 3, 4, 21),
 (1, 2, 3, 4, 22),
 (1, 2, 3, 4, 23),
 (1, 2, 3, 4, 24),
 (1, 2, 3, 4, 25),
 (1, 2, 3, 4, 26),
 (1, 2, 3, 4, 27),
 (1, 2, 3, 4, 28),
 (1, 2, 3, 4, 29),
 (1, 2, 3, 4, 30),
 (1, 2, 3, 4, 31),
 (1, 2, 3, 4, 32),
 (1, 2, 3, 4, 33),
 (1, 2, 3, 4, 34),
 (1, 2, 3, 4, 35),
 (1, 2, 3, 4, 36),
 (1, 2, 3, 4, 37),
 (1, 2, 3, 4, 38),
 (1, 2, 3, 4, 39),
 (1, 2, 3, 4, 40),
 (1, 2, 3, 4, 41),
 (1, 2, 3, 4, 42),
 (1, 2, 3, 4, 43),
 (1, 2, 3, 5, 6),
 (1, 2, 3, 5, 7),
 (1, 2, 3, 5, 8),
 (1, 2, 3, 5, 9),
 (1, 2, 3, 5, 10),
 (1, 2, 3, 5, 11),
 (1, 2, 3, 5, 12),
 (1, 2, 3, 5, 13),
 (1, 2, 3, 5, 14),
 (1, 2, 3, 5, 15),
 (1, 2, 3, 5, 16),
 (1, 2, 3, 5, 17),
 (1, 2, 3, 5, 18),
 (1, 2, 3, 5, 19),
 (

In [97]:
import numpy as np
from itertools import combinations, chain
from scipy.special import comb

def comb_index(n, k):
    count = comb(n, k, exact=True)
    index = np.fromiter(chain.from_iterable(combinations(range(n), k)), 
                        int, count=count*k)
    return index.reshape(-1, k)

In [132]:
total_idx = comb_index(52, 5)

In [134]:
hand_combos = deck_arr[total_idx, :]

In [139]:
hand = hand_combos[0]

In [169]:
hand[:, 0]

array([2, 2, 2, 2, 3])

In [170]:
np.sort??

In [234]:
scores = np.array([hand_score(hand_combos[i, :, :]) for i in range(10000)])

In [257]:
import timeit

start = timeit.default_timer()
hand_scores = np.apply_along_axis(func1d=hand_score,
    arr=hand_combos.reshape(-1, 10),
    axis=1)
end = timeit.default_timer()
print(end - start)

193.9274779009993


In [ ]:
hand_scores

In [ ]:
import timeit

start = timeit.default_timer()
hand_scores = parallel_apply_along_axis(func1d=hand_score,
    arr=hand_combos.reshape(-1, 10),
    axis=1)
end = timeit.default_timer()
print(end - start)

In [263]:
import multiprocessing
def parallel_apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """
    Like numpy.apply_along_axis(), but takes advantage of multiple
    cores.
    """        
    # Effective axis where apply_along_axis() will be applied by each
    # worker (any non-zero axis number would work, so as to allow the use
    # of `np.array_split()`, which is only done on axis 0):
    effective_axis = 1 if axis == 0 else axis
    if effective_axis != axis:
        arr = arr.swapaxes(axis, effective_axis)

    # Chunks for the mapping (only a few chunks):
    chunks = [(func1d, effective_axis, sub_arr, args, kwargs)
              for sub_arr in np.array_split(arr, multiprocessing.cpu_count())]

    pool = multiprocessing.Pool()
    individual_results = pool.map(unpacking_apply_along_axis, chunks)
    # Freeing the workers:
    pool.close()
    pool.join()

    return np.concatenate(individual_results)

def unpacking_apply_along_axis(allargs):
    """
    Like numpy.apply_along_axis(), but with arguments in a tuple
    instead.

    This function is useful with multiprocessing.Pool().map(): (1)
    map() only handles functions that take a single argument, and (2)
    this function can generally be imported from a module, as required
    by map().
    """
    (func1d, axis, arr, args, kwargs) = all_args

In [249]:
def hand_score(hand):
        
    hand_values = np.sort(hand[[0,2,4,6,8]])[::-1]
    hand_suits = np.sort(hand[[1,3,5,7,9]])
    
    return assign_rank(hand_suits, hand_values)
    

def suit_check(hand_suits):
    return np.all(hand_suits[0] == hand_suits)

def straight_check(hand_values):
    return np.all(hand_values == np.arange(hand_values[0], hand_values[0] - 5, -1)) | \
           np.all(hand_values == np.array([14,5,4,3,2]))

def assign_rank(hand_suits, hand_values):
    
    is_suited = suit_check(hand_suits)
    is_straight = straight_check(hand_values)
    
    if np.all(hand_values == np.array([2,3,4,5,14])):
        hand_values = np.array([1,2,3,4,5])
    
    if is_suited and is_straight and hand_values[0] == 10:
        rank = np.concatenate([np.array([10]),  hand_values])
    elif is_suited and is_straight:
        rank = np.concatenate([np.array([9]),  hand_values])
    elif is_suited:
        rank = np.concatenate([np.array([6]),  hand_values])
    elif is_straight:
        rank = np.concatenate([np.array([5]),  hand_values])
    else:
        val, count = np.unique(hand_values, return_counts=True)
        rank = np.concatenate(
                [np.array([rank_dict((max(count), min(count), len(count)))]),
                 val[count.argsort()]])
        
#     print(rank)
    return assign_hand_value(rank)

def assign_hand_value(hand_array):
        return int(np.sum(np.exp2(np.arange(20, 20 - len(hand_array) * 4, -4)) * hand_array))
    
def rank_dict(max_min_len):
        rank_params_dict = {(4, 1, 2): 8, (3, 2, 2): 7, (3, 1, 3): 4, (2, 1, 3): 3, (2, 1, 4): 2, (1, 1, 5): 1}
        return rank_params_dict[max_min_len]
    
    

In [166]:
straight_check(np.array([2,3,4,5,6]))

True

In [156]:
    """
    Assigns the rank of the hand
    """

    def assign_rank(self, hand):

        suit_check = self.suit_check(hand)
        straight_check = self.straight_check(hand)
        hand_values = self._hand_values(hand)
        
        if hand_values == [2,3,4,5,14]:
            hand_values = [1,2,3,4,5]
        ### We could use straight check and suit check to quickly determine certain card types
        if suit_check and straight_check and hand_values[0] == 10:
            return np.array([10] + hand_values[::-1])
        elif suit_check and straight_check:
            return np.array([9] + hand_values[::-1])
        elif suit_check:
            return np.array([6] + hand_values[::-1])
        elif straight_check:
            return np.array([5] + hand_values[::-1])

        ### Assign all other categories
        val, count = np.unique(hand_values, return_counts=True)
#         print(val, count)
        return np.array([self._rank_dict((max(count), min(count), len(count)))] + list(val[count.argsort()])[::-1])
    
    def assign_value(self, hand_array):
        return int(np.sum(np.exp2(np.arange(20, 20 - len(hand_array) * 4, -4)) * hand_array))

    def _rank_dict(self, max_min_len):
        rank_params_dict = {(4, 1, 2): 8, (3, 2, 2): 7, (3, 1, 3): 4, (2, 1, 3): 3, (2, 1, 4): 2, (1, 1, 5): 1}
        return rank_params_dict[max_min_len]

IndentationError: unexpected indent (<ipython-input-156-457b0b48a1cd>, line 2)

In [124]:
np.concatenate(
    [deck_arr[[3, 5]], hand_combos], axis=1)

ValueError: all the input arrays must have same number of dimensions

In [129]:
deck_arr[[3, 5], :].shape

(2, 2)

In [130]:
hand_combos

array([[[ 2,  0],
        [ 2,  1],
        [ 2,  2]],

       [[ 2,  0],
        [ 2,  1],
        [ 2,  3]],

       [[ 2,  0],
        [ 2,  1],
        [ 3,  0]],

       ...,

       [[14,  0],
        [14,  1],
        [14,  3]],

       [[14,  0],
        [14,  2],
        [14,  3]],

       [[14,  1],
        [14,  2],
        [14,  3]]])